In [ ]:
import torch
import torchsummary
import os
import sys
import pytorch_lightning as pl
import torch.optim as optim
import torch.nn as nn

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(ROOT_DIR)

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(ROOT_DIR)

from utils import helper, qai_hub_jobs, tfhelper
import training
import casvit.rcvit as rcvit

c:\Users\stale\Desktop\LPCV\LPCV_2025_ETF_Amigo\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\stale\Desktop\LPCV\LPCV_2025_ETF_Amigo\.venv\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\stale\Desktop\LPCV\LPCV_2025_ETF_Amigo\.venv\Lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [2]:
def getBaseModel():
    modelBase = rcvit.rcvit_t()

    weights = torch.randn([64])

    modelBase.dist = False
    modelBase.head = nn.Linear(512, 64) # Menja classifier head da bude 64

    return modelBase

In [3]:
import training.lightning_model
import training.lightning_train_function

modelBase = getBaseModel()

lightningModel = training.lightning_model.LightningModel.load_from_checkpoint(r"../../finalLightningCheckpoint/FINAL_MODEL-epoch=54-train_loss=0.6094-val_loss=0.8717.ckpt", model = modelBase)

model = lightningModel.model
model.eval()

Init device
cpu


C:\Users\stale\Desktop\LPCV\LPCV_2025_ETF_Amigo\src\training\lightning_model.py:20: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\stale\Desktop\LPCV\LPCV_2025_ETF_Amigo\src\training\lightning_model.py:25: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).


RCViT(
  (patch_embed): Sequential(
    (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (network): ModuleList(
    (0): Sequential(
      (0): AdditiveBlock(
        (local_perception): LocalIntegration(
          (network): Sequential(
            (0): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
            (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=96)
            (3): GELU(approximate='none')
            (4): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
          )
        )
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, aff

In [4]:
from dataset import DatasetReader
from dataset.utils import GLOBAL_CLASSES
from torchvision import transforms
from torch.utils.data import DataLoader

In [6]:
import torch.nn as nn
import torchvision.ops as ops
import torchvision.transforms as transforms

import torchvision.transforms.functional as F

class Normalized(torch.nn.Module):

    def __init__(self, network):
        super(Normalized, self).__init__()
        self.model = network
        self.preprocess = transforms.Compose([
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def forward(self, img):
        # Pass the preprocessed image through the model
        return self.model(self.preprocess(img))

modelOriginal = model

model = Normalized(modelOriginal)

In [9]:
import qai_hub

input_shape = (1, 3, 224, 224)

dummy_input  = torch.randn(1, 3, 224, 224)
pt_model = torch.jit.trace(model.cpu().eval(), dummy_input)

In [10]:
compile_job = qai_hub.submit_compile_job(
    pt_model,
    name="Final Model epoch 54", # Replace with your model name
    device=qai_hub.Device("Snapdragon 8 Elite QRD"),
    input_specs=dict(image=input_shape)
)

compile_job.modify_sharing(add_emails=['lowpowervision@gmail.com']) ## Share your model for submission

Uploading tmpet4qdwh8.pt


100%|██████████| 82.8M/82.8M [00:10<00:00, 8.43MB/s]


Scheduled compile job (jgk73x4vp) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jgk73x4vp/



In [ ]:
profile_job     = qai_hub.submit_profile_job(compile_job.get_target_model(), device=qai_hub.Device("Snapdragon 8 Elite QRD"), name = "Profile job")

Scheduled profile job (jp829jezp) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp829jezp/

